Frozen Lake
===
The goal of this computer assignment is to get familiar with OpenAI Gym, implement value iteration and policy iteration.

Problem Description
---
OpenAI Gym is a toolkit for developing and comparing reinforcement learning algorithms. It supports teaching agents everything from walking to playing games like Pong or Pinball. For more information visit https://gym.openai.com.

In this computer assigment, you'll get familiar with Frozen Lake environment and implement value and policy iteration algorithms. Frozen Lake is an environment where the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. For more information please visit https://gym.openai.com/envs/FrozenLake8x8-v0/.

Your Job
---
1. Get started with gym by following the steps here https://gym.openai.com/docs/.
2. Read https://gym.openai.com/envs/FrozenLake8x8-v0/ to get familiar with the environment, states, reward function, etc.
3. Implement the $\texttt{value_iteration}$ function below.
4. Implement the $\texttt{policy_iteration}$ function below.
5. Answer the questions (By double click on the cell you can edit the cell and put your answer below each question).

In [5]:
from __future__ import print_function
from __future__ import division

import numpy as np
from numpy import linalg as LA
import gym
from gym import wrappers
import time

def run_episode(env, policy, gamma, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma=gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma, epsilon=1e-20, max_iterations=100000):
    """
    This function implements value iteration algorithm for the infinite
    horizon discounted MDPs. If the sup norm of v_k - v_{k-1} is less than
    epsilon or number of iterations reaches max_iterations, it should return
    the value function.
    """
    start = time.time()
    v = np.zeros(env.nS)  # initialize value-function
    ########################### Your Code Here ###########################
    # Hint: see implementation of extract_policy
    
    for ite in np.arange(max_iterations):
    
        if ite != 0:
            v = v_upd_whole

        v_upd_whole = np.zeros(env.nS)
        for s in np.arange(env.nS):
            v_collec = np.zeros(env.action_space.n)
            for a in np.arange(env.action_space.n):
                v_loc = 0
                for jmps in env.P[s][a]:
                    p,s_next,r,_ = jmps
                    v_loc = v_loc + p*r + gamma*p*v[s_next]

                v_collec[a] = v_loc


            v_upd_ind = np.max(v_collec)
            v_upd_whole[s] = v_upd_ind
            
        if LA.norm(v_upd_whole-v,np.inf) < epsilon:
            v = v_upd_whole
            eff_ite = ite + 1
            print('Total iterations needed for convergence were :',eff_ite)
            break
        
        if ite == max_iterations - 1:
            v = v_upd_whole
            tot_ite = ite + 1
            print('Total iterations, without convergence, were the maximum iterations :',tot_ite)

       
     
    
    
    
    
    
    ########################### End of your code #########################
    end = time.time()
    print("Value iteration took {0} seconds.".format(end - start))
    return v

if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_v = value_iteration(env, gamma);
        policy = extract_policy(optimal_v, gamma)
        policy_score = evaluate_policy(env, policy, gamma, n=1000)
        print('Average score = ', policy_score)

---------- Gamma=0.9 ----------
Total iterations needed for convergence were : 315
Value iteration took 0.3606681823730469 seconds.
Average score =  0.0060644523272935565
---------- Gamma=0.95 ----------
Total iterations needed for convergence were : 495
Value iteration took 0.6319043636322021 seconds.
Average score =  0.048616381696941935
---------- Gamma=0.99 ----------
Total iterations needed for convergence were : 1128
Value iteration took 1.4338624477386475 seconds.
Average score =  0.40879407428412834
---------- Gamma=0.9999 ----------
Total iterations needed for convergence were : 2337
Value iteration took 2.8529796600341797 seconds.
Average score =  0.8756445024265985
---------- Gamma=1 ----------
Total iterations needed for convergence were : 2357
Value iteration took 3.0036962032318115 seconds.
Average score =  0.878


Policy Iteration
---

In [22]:
def compute_policy_v(env, policy, gamma):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma, max_iterations=100000):
    """
    This function implements policy iteration algorithm.
    """
    start = time.time()
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy
    ########################### Your Code Here ###########################
    
    
    for ite in np.arange(max_iterations):
        
        v_upd_whole = compute_policy_v(env,policy,gamma)           # only major algorithmic modification
        policy_old = np.copy(policy)
        v = v_upd_whole
        for s in np.arange(env.nS):
            v_collec = np.zeros(env.action_space.n)
            for a in np.arange(env.action_space.n):
                v_loc = 0
                for jmps in env.P[s][a]:
                    p,s_next,r,_ = jmps
                    v_loc = v_loc + p*r + gamma*p*v[s_next]

                v_collec[a] = v_loc
                
            
            policy[s] = np.argmax(v_collec)
            
            
        

        if np.array_equal(policy_old,policy):
            eff_ite = ite + 1
            print('Total iterations needed for convergence were :',eff_ite)
            break
        
        if ite == max_iterations - 1:
            tot_ite = ite + 1
            print('Total iterations, without convergence, were the maximum iterations :',tot_ite)

        
        
    
    
    
    
    
    
    
    
    
    ########################### End of your code #########################
    end = time.time()
    print("Policy iteration took {0} seconds.".format(end - start))
    return policy


if __name__ == '__main__':
    np.random.seed(1111)
    env_name  = 'FrozenLake8x8-v0'
    for gamma in [.9, .95, .99, .9999, 1]:
        print("-"*10, "Gamma={0}".format(gamma) ,"-"*10)
        env = gym.make(env_name)
        optimal_policy = policy_iteration(env, gamma=gamma)
        scores = evaluate_policy(env, optimal_policy, gamma=gamma)
        print('Average scores = ', np.mean(scores))

---------- Gamma=0.9 ----------
Total iterations needed for convergence were : 5
Policy iteration took 0.2047128677368164 seconds.
Average scores =  0.0062415895498807135
---------- Gamma=0.95 ----------
Total iterations needed for convergence were : 3
Policy iteration took 0.2377605438232422 seconds.
Average scores =  0.04831133610155477
---------- Gamma=0.99 ----------
Total iterations needed for convergence were : 8
Policy iteration took 1.3672118186950684 seconds.
Average scores =  0.3862772551379816
---------- Gamma=0.9999 ----------
Total iterations needed for convergence were : 12
Policy iteration took 4.202902793884277 seconds.
Average scores =  0.8814389744647637
---------- Gamma=1 ----------
Total iterations needed for convergence were : 6
Policy iteration took 2.4776272773742676 seconds.
Average scores =  0.93


Questions
--- 

#### 1. How many iterations did it take for the value iteration to converge? How about policy iteration?
Value iterations for me converged in:
315, 495, 1128, 2337, 2357 
Around 500-1500 iterations for Value iterations could thus be expected
As gamma, increased no. of iterations needed for convergence also increased


Policy iterations for me converged in:
5,3,8,12,6 iterations
Thus around 5-10 iterations were needed on average for the policy iteration algorithm to converge, however time consumed per iteration, as expected, was much larger.


#### 2. How much time did it take for the value iteration to converge? How about the policy iteration?
Value iterations converged in:
0.36,0.63,1.43,2.85,3.00 seconds
Again as gamma increased, time to converge also increased.

Policy iterations converged in:
0.2,0.23,1.36,4.2,2.44 seconds
As such, time for policy iterations wasn't much smaller than value iteration despite the no. of value iterations being much much larger than the number of policy iterations


#### 3. Which algorithm is faster? Why?
Current run for me gave me results where both the algorithms were similar in terms of overall run times. However, as such, policy iterations generally win over value iterations, as discussed in the lecture, for complicated cases. Since, it also ends up giving the policy directly which the value iteration doesn't, policy iterations could be more favourable in general.

Policy iterations are smaller in number than value iterations, however, policy iterations, per iteration, require more runtime than the value iteration. 

Since, the policy iteration converges with equality in finite steps, for complicated scenarios it could be picked since it could perform better in terms of speed


#### 4. How does the average score change as $\gamma$ gets closer to 1? Why?
For Value iteration:
As the gamma value increased, for me, the average score also increased and then at 1 it reached its maximum

For Policy iteration:
The trend here was also similar. As the value of gamma increased the average score increased.


Reason: The overall reward function for each action and state, has, somewhat increased. Thus as gamma increases, one can expect the scores to increase, since overall each step reward also correspondingly increases with the increase in gamma.